# Mining testimonial fragments of the Holocaust

**Experience domain:**

### Load the necessary libraries

In [1]:
import sys; sys.path.insert(0, '..')
import itertools

In [2]:
import get_topic_model_concordance as topic_concordancer
from utils import blacklab, db, text
mongo = db.get_db()

In [3]:
%config Completer.use_jedi = False
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import random

### Helper functions

In [4]:
def create_contextual_query(lemmas,context_length=50):
    permutations = itertools.permutations(lemmas,len(lemmas))
    final_result = []
    for element in list(permutations):
        temp_result = []
        for el in element:
            temp_result.append('[lemma="'+el+'"]')
        temp_result = '('+('[]{0,'+str(context_length)+'}').join(temp_result)+')'
        final_result.append(temp_result)
    final_result = '|'.join(final_result)
    return final_result
        
        
            

In [5]:
from utils import blacklab, db, text
import requests
import json
def find_sentence_id(label):
    props = {'annotators': 'tokenize'}

    # set the encoding of the annotator
    requests.encoding = 'utf-8'
    # make a request
    r = requests.post('http://localhost:9000/', params={'properties':
                      json.dumps(props)},
                      data=label.encode('utf-8'))
    result = json.loads(r.text, encoding='utf-8')
    query = []
    for i, token in enumerate(result['tokens']):

        if ('...'in token['word'] and ((i == 0) or
           i == len(result['tokens']) - 1)):
            continue
        elif ('...'in token['word']):
            query.append('[]{0,50}')
        elif ('-'in token['word']):
            query.append('[]{0,3}')
        elif ("n't"in token['word']):
            query.append('[]')
        elif ("'re"in token['word']):
            query.append('[]')
        elif ("?"in token['word']):
            query.append('[]')
        elif ("."in token['word']):
            query.append('[]')
        elif ("'s"in token['word']):
            query.append('[]')
        elif (","in token['word']):
            query.append('[]')
        else:
            query.append('["' + token['word'] + '"]')

    query = ' '.join(query)
    try:
        sentence = blacklab.search_blacklab(query, window=0,
                                            lemma=False,
                                            include_match=True)
        token_end = sentence[0]['token_end']
        token_start = sentence[0]['token_start']
        print (sentence[0])
        mongo = db.get_db()
        results = mongo.tokens.find({'testimony_id':
                                    sentence[0]['testimony_id']},
                                    {'_id': 0})
        tokens = list(results)[0]['tokens']
        sentenceStart = tokens[token_start]['sentence_index']
        sentenceEnd = tokens[token_end]['sentence_index']
        originalsentence = sentence[0]['complete_match']
        return (sentenceStart,sentenceEnd,sentence[0]['testimony_id'])
    except:
        print("The following query returned a null result")
        print(query)
        
            


In [6]:
def create_parent_node(label):
    """Generate a root node for a tree structure."""
    testimony_id = random.randint(1, 20)
    node = {}
    node['label'] = label
    fragment = {'label': label,
                'essay_id': random.randint(1, 20),
                'tree': get_node(testimony_id, node, is_parent=True)}
    fragment['tree']['label'] = label

    return fragment

In [7]:
def get_node(testimony_id, node, is_parent=False):
    """Generate a parent or leaf node for a tree structure."""
    if is_parent:
        return {
            'label': node['label'],
            'testimony_id': random.randint(1, 20),
            'media_index': random.randint(1, 20),
            'media_offset': random.randint(1, 20),
            'start_sentence_index': random.randint(1, 20),
            'end_sentence_index': random.randint(1, 20),
            'children': [], }
    else:
        return {'label': node['label'],
                'testimony_id': node['testimony_id'],
                'media_index': float(node['media_index']),
                'media_offset': float(node['media_offset']),
                'start_sentence_index': float(node['start_sentence_index']),
                'end_sentence_index': float(node['end_sentence_index']),
                'children': [], }

In [8]:
def check_if_main_node_exist(node):
    results = mongo.fragments.find({'label':node},{'_id': 0})
    if len(results[0])==0:
        return False
    else:
        return True

In [9]:
def add_main_node(label):
    mongo.fragments.insert(create_parent_node(label))

In [10]:
def delete_main_node(label):
    mongo.fragments.delete_one({'label':label})

In [11]:
def add_testimonial_fragments(fragments):
    if check_if_main_node_exist(fragments['main_node']):
        results = mongo.fragments.find({'label':fragments['main_node']},{'_id':0})[0]
        mid_nodes = [element['label'] for element in results['tree']['children']]
        if fragments['mid_node'] in mid_nodes:
            print ("mid node exists cannot be added")
        else:
            
            mid_node = get_node('r',{'label':fragments['mid_node']},is_parent=True)
            for fragment in fragments['fragments']:
                leaf = get_node(fragment['testimony_id'],fragment)
                mid_node['children'].append(leaf)
            results['tree']['children'].append(mid_node)
            mongo.fragments.replace_one({'label':fragments['main_node']},results)

### Add the main node

In [40]:
main_node = "force"
#delete_main_node(main_node)
add_main_node(main_node)

/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


### Set up the query

In [12]:
query = '[lemma="force" &pos="V.*"] [lemma!="labor"]'

In [17]:
result = topic_concordancer.main(query,window=20,topicn=25)

Training lda model began
http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5Blemma%3D%22force%22+%26pos%3D%22V.%2A%22%5D+%5Blemma%21%3D%22labor%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=20
training of gensim corpus began
gensim corpus done


### Print the key topics

In [16]:
for i,element in enumerate(result['topic_documents']):
    print (i)
    topic_words =  element['topic_words'][1]
    print (topic_words)
    print ('\n')

0
0.444*"do_not" + 0.224*"family" + 0.158*"stay" + 0.092*"street" + 0.026*"go_back" + 0.020*"order" + 0.015*"understand" + 0.010*"start" + 0.005*"Poland" + 0.005*"class"


1
0.508*"Jews" + 0.228*"wear" + 0.101*"star" + 0.069*"group" + 0.053*"hang" + 0.021*"what_happen" + 0.011*"force_labor" + 0.005*"cry" + 0.005*"Hitler" + 0.000*"allow"


2
0.470*"give" + 0.131*"open" + 0.089*"food" + 0.083*"house" + 0.048*"start" + 0.048*"order" + 0.036*"law" + 0.024*"mother" + 0.018*"stand" + 0.018*"german"


3
0.278*"could_not" + 0.128*"join" + 0.100*"train" + 0.094*"job" + 0.089*"find" + 0.072*"push" + 0.056*"farm" + 0.056*"my_sister" + 0.050*"one_day" + 0.022*"a_lot"


4
0.343*"make" + 0.172*"walk" + 0.157*"feel" + 0.071*"force_labor" + 0.071*"good" + 0.061*"transport" + 0.040*"stand" + 0.040*"suppose" + 0.010*"people_who" + 0.010*"eat"


5
0.438*"time" + 0.203*"thing" + 0.062*"factory" + 0.057*"kind_of" + 0.057*"woman" + 0.052*"come_back" + 0.031*"story" + 0.021*"remember" + 0.021*"my_parent" + 0

### Analyze documents

In [ ]:
i=0
for text in result['topic_documents'][i]['texts'][0:25]:
    print (text['matched_text_words'])
    print ('\n')

## Testimonial fragments

### 1.  

In [41]:
query = '[lemma="force"] []{0,16} [lemma="watch"] []{0,15} [lemma="hang.*"]'

In [42]:
query = '[lemma="have"] ["to"] [lemma="watch"] []{0,15} [lemma="hang.*"]'

In [43]:
domain_term = "watch hanging"

In [44]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [45]:
fragment_1 = {}
fragment_1['original_sentence'] = "there was a central place where we all were forced to gather and to watch the hangings."
fragment_1['label']="(..) there was a central place where we all were forced to gather and to watch the hangings."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22there%22%5D+%5B%22was%22%5D+%5B%22a%22%5D+%5B%22central%22%5D+%5B%22place%22%5D+%5B%22where%22%5D+%5B%22we%22%5D+%5B%22all%22%5D+%5B%22were%22%5D+%5B%22forced%22%5D+%5B%22to%22%5D+%5B%22gather%22%5D+%5B%22and%22%5D+%5B%22to%22%5D+%5B%22watch%22%5D+%5B%22the%22%5D+%5B%22hangings%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'there was a central place where we all were forced to gather and to watch the hangings . ', 'right': '', 'complete_match': 'there was a central place where we all were forced to gather and to watch the hangings . ', 'testimony_id': 'irn504408', 'shelfmark': ['USHMM RG-50.030*0203'], 'token_start': 10874, 'token_end': 10892}


In [46]:
fragment_2 = {}
fragment_2['original_sentence'] = "And, you know, most of the people that tried to run away were hanged the next day. We didn’t -- and we were forced to watch the hanging, of course."
fragment_2['label']="(..)people that tried to run away were hanged the next day. (..)we were forced to watch the hanging(..)."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%5D+%5B%22you%22%5D+%5B%22know%22%5D+%5B%5D+%5B%22most%22%5D+%5B%22of%22%5D+%5B%22the%22%5D+%5B%22people%22%5D+%5B%22that%22%5D+%5B%22tried%22%5D+%5B%22to%22%5D+%5B%22run%22%5D+%5B%22away%22%5D+%5B%22were%22%5D+%5B%22hanged%22%5D+%5B%22the%22%5D+%5B%22next%22%5D+%5B%22day%22%5D+%5B%5D+%5B%22We%22%5D+%5B%22did%22%5D+%5B%5D+%5B%5D%7B0%2C3%7D+%5B%22and%22%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22forced%22%5D+%5B%22to%22%5D+%5B%22watch%22%5D+%5B%22the%22%5D+%5B%22hanging%22%5D+%5B%5D+%5B%22of%22%5D+%5B%22course%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And , you know , most of the people that tried to run away were hanged the next day . We did n’t -- and we were forced to watch the hanging , of course . ', 'right': '', 'complete_match': 'And , you know , most of the people that tried to run away were hanged the next day . We did n’t -- and we were 

In [47]:
fragment_3 = {}
fragment_3['original_sentence'] = "my brother and two of his friends were hanged in front of 15,000 prisoners who were forced to stand there and to watch the show"
fragment_3['label']="(..)my brother and two of his friends were hanged in front of 15,000 prisoners who were forced to stand there and to watch the show(..)."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22my%22%5D+%5B%22brother%22%5D+%5B%22and%22%5D+%5B%22two%22%5D+%5B%22of%22%5D+%5B%22his%22%5D+%5B%22friends%22%5D+%5B%22were%22%5D+%5B%22hanged%22%5D+%5B%22in%22%5D+%5B%22front%22%5D+%5B%22of%22%5D+%5B%5D+%5B%22prisoners%22%5D+%5B%22who%22%5D+%5B%22were%22%5D+%5B%22forced%22%5D+%5B%22to%22%5D+%5B%22stand%22%5D+%5B%22there%22%5D+%5B%22and%22%5D+%5B%22to%22%5D+%5B%22watch%22%5D+%5B%22the%22%5D+%5B%22show%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'my brother and two of his friends were hanged in front of 15,000 prisoners who were forced to stand there and to watch the show ', 'right': '', 'complete_match': 'my brother and two of his friends were hanged in front of 15,000 prisoners who were forced to stand there and to watch the show ', 'testimony_id': 'usc_shoah_17135', 'shelfmark': ['USC 17135'], 'token_start': 47576, 'token_end': 47601}


In [49]:
fragment_4 = {}
fragment_4['original_sentence'] = "he and his two sons were hung"
fragment_4['label']= "(..)he and his two sons were hung [hanged] and I remember that we were all forced to go and watch it."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22he%22%5D+%5B%22and%22%5D+%5B%22his%22%5D+%5B%22two%22%5D+%5B%22sons%22%5D+%5B%22were%22%5D+%5B%22hung%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'he and his two sons were hung ', 'right': '', 'complete_match': 'he and his two sons were hung ', 'testimony_id': 'irn504725', 'shelfmark': ['USHMM RG-50.030*0236'], 'token_start': 7219, 'token_end': 7226}


In [50]:
fragment_5 = {}
fragment_5['original_sentence'] = "We all had to watch by the appell how the man was hanged."
fragment_5['label']= "We all had to watch by the appell how the man was hanged."
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22We%22%5D+%5B%22all%22%5D+%5B%22had%22%5D+%5B%22to%22%5D+%5B%22watch%22%5D+%5B%22by%22%5D+%5B%22the%22%5D+%5B%22appell%22%5D+%5B%22how%22%5D+%5B%22the%22%5D+%5B%22man%22%5D+%5B%22was%22%5D+%5B%22hanged%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'We all had to watch by the appell how the man was hanged . ', 'right': '', 'complete_match': 'We all had to watch by the appell how the man was hanged . ', 'testimony_id': 'irn504693', 'shelfmark': ['USHMM RG-50.030*0199'], 'token_start': 7361, 'token_end': 7375}


In [51]:
add_testimonial_fragments(fragments)

### 2.  

In [52]:
lemmas = ["force","move"]

In [53]:
query = create_contextual_query(lemmas,context_length=10)
print (query)

([lemma="force"][]{0,10}[lemma="move"])|([lemma="move"][]{0,10}[lemma="force"])


In [54]:
domain_term = "leave home"

In [55]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [56]:
fragment_1 = {}
fragment_1['original_sentence'] = "And uh as I told you, my my garden hold held most of my wonderful childhood memories and uh fairly soon...I would say probably in early 1940, uh there was a sign on it that no Jews or dogs were permitted in the garden, so we naturally did not go but on the morning on which we were forced to leave our home, I jumped over the fence and went to the garden and I ran around."
fragment_1['label']="(..)my garden held most of my wonderful childhood memories but on the morning on which we were forced to leave our home, I (..) went to the garden(..)"
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22uh%22%5D+%5B%22as%22%5D+%5B%22I%22%5D+%5B%22told%22%5D+%5B%22you%22%5D+%5B%5D+%5B%22my%22%5D+%5B%22my%22%5D+%5B%22garden%22%5D+%5B%22hold%22%5D+%5B%22held%22%5D+%5B%22most%22%5D+%5B%22of%22%5D+%5B%22my%22%5D+%5B%22wonderful%22%5D+%5B%22childhood%22%5D+%5B%22memories%22%5D+%5B%22and%22%5D+%5B%22uh%22%5D+%5B%22fairly%22%5D+%5B%22soon%22%5D+%5B%5D%7B0%2C50%7D+%5B%22I%22%5D+%5B%22would%22%5D+%5B%22say%22%5D+%5B%22probably%22%5D+%5B%22in%22%5D+%5B%22early%22%5D+%5B%221940%22%5D+%5B%5D+%5B%22uh%22%5D+%5B%22there%22%5D+%5B%22was%22%5D+%5B%22a%22%5D+%5B%22sign%22%5D+%5B%22on%22%5D+%5B%22it%22%5D+%5B%22that%22%5D+%5B%22no%22%5D+%5B%22Jews%22%5D+%5B%22or%22%5D+%5B%22dogs%22%5D+%5B%22were%22%5D+%5B%22permitted%22%5D+%5B%22in%22%5D+%5B%22the%22%5D+%5B%22garden%22%5D+%5B%5D+%5B%22so%22%5D+%5B%22we%22%5D+%5B%22naturally%22%5D+%5B%22did%22%5D+%5B%22not%22%5D+%5B%22go%22%5D+%5B%22but%22%5D+%5B%22on%22%5D+%5B%22the%22%5D+%5

In [57]:
fragment_2 = {}
fragment_2['original_sentence'] = "that specific moment in which we were forced to leave our home became embedded in my art."
fragment_2['label']="(..)that specific moment in which we were forced to leave our home became embedded in my art."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22that%22%5D+%5B%22specific%22%5D+%5B%22moment%22%5D+%5B%22in%22%5D+%5B%22which%22%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22forced%22%5D+%5B%22to%22%5D+%5B%22leave%22%5D+%5B%22our%22%5D+%5B%22home%22%5D+%5B%22became%22%5D+%5B%22embedded%22%5D+%5B%22in%22%5D+%5B%22my%22%5D+%5B%22art%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'that specific moment in which we were forced to leave our home became embedded in my art . ', 'right': '', 'complete_match': 'that specific moment in which we were forced to leave our home became embedded in my art . ', 'testimony_id': 'irn538186', 'shelfmark': ['USHMM RG-50.926*0001'], 'token_start': 1246, 'token_end': 1264}


In [58]:
fragment_3 = {}
fragment_3['original_sentence'] = "they told the Jews have to leave their towns...their towns and homes and belongings."
fragment_3['label']="they told the Jews have to leave their towns...their towns and homes and belongings."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22they%22%5D+%5B%22told%22%5D+%5B%22the%22%5D+%5B%22Jews%22%5D+%5B%22have%22%5D+%5B%22to%22%5D+%5B%22leave%22%5D+%5B%22their%22%5D+%5B%22towns%22%5D+%5B%5D%7B0%2C50%7D+%5B%22their%22%5D+%5B%22towns%22%5D+%5B%22and%22%5D+%5B%22homes%22%5D+%5B%22and%22%5D+%5B%22belongings%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'they told the Jews have to leave their towns ... their towns and homes and belongings . ', 'right': '', 'complete_match': 'they told the Jews have to leave their towns ... their towns and homes and belongings . ', 'testimony_id': 'irn504553', 'shelfmark': ['USHMM RG-50.030*0055'], 'token_start': 2283, 'token_end': 2300}


In [59]:
fragment_4 = {}
fragment_4['original_sentence'] = "And this was the worst thing that would happen til now. We had to leave our home."
fragment_4['label']= "And this was the worst thing that would happen til now. We had to leave our home."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22this%22%5D+%5B%22was%22%5D+%5B%22the%22%5D+%5B%22worst%22%5D+%5B%22thing%22%5D+%5B%22that%22%5D+%5B%22would%22%5D+%5B%22happen%22%5D+%5B%22til%22%5D+%5B%22now%22%5D+%5B%5D+%5B%22We%22%5D+%5B%22had%22%5D+%5B%22to%22%5D+%5B%22leave%22%5D+%5B%22our%22%5D+%5B%22home%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And this was the worst thing that would happen til now . We had to leave our home . ', 'right': '', 'complete_match': 'And this was the worst thing that would happen til now . We had to leave our home . ', 'testimony_id': 'irn504693', 'shelfmark': ['USHMM RG-50.030*0199'], 'token_start': 17828, 'token_end': 17847}


In [60]:
fragment_5 = {}
fragment_5['original_sentence'] = "all the Jewish people had to move out of their homes and go to live in that area."
fragment_5['label']= "(..) all the Jewish people had to move out of their homes and go to live in that area."
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22all%22%5D+%5B%22the%22%5D+%5B%22Jewish%22%5D+%5B%22people%22%5D+%5B%22had%22%5D+%5B%22to%22%5D+%5B%22move%22%5D+%5B%22out%22%5D+%5B%22of%22%5D+%5B%22their%22%5D+%5B%22homes%22%5D+%5B%22and%22%5D+%5B%22go%22%5D+%5B%22to%22%5D+%5B%22live%22%5D+%5B%22in%22%5D+%5B%22that%22%5D+%5B%22area%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'all the Jewish people had to move out of their homes and go to live in that area . ', 'right': '', 'complete_match': 'all the Jewish people had to move out of their homes and go to live in that area . ', 'testimony_id': 'irn504617', 'shelfmark': ['USHMM RG-50.030*0123'], 'token_start': 542, 'token_end': 561}


In [61]:
add_testimonial_fragments(fragments)

### 3.  

In [62]:
lemmas = ["force","walk"]

In [63]:
query = create_contextual_query(lemmas,context_length=10)
print (query)

([lemma="force"][]{0,10}[lemma="walk"])|([lemma="walk"][]{0,10}[lemma="force"])


In [66]:
query = '([lemma="force"][lemma=".*self"] []{0,10}[lemma="walk"])|([lemma="walk"][]{0,10}[lemma="force"])'

In [67]:
domain_term = "myself walking"

In [68]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [69]:
fragment_1 = {}
fragment_1['original_sentence'] = " I-- I forced-- I forced-- and then I forced myself walking. I forced."
fragment_1['label']=" I-- I forced-- I forced-- and then I forced myself walking. I forced."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22I%22%5D+%5B%5D%7B0%2C3%7D+%5B%22I%22%5D+%5B%22forced%22%5D+%5B%5D%7B0%2C3%7D+%5B%22I%22%5D+%5B%22forced%22%5D+%5B%5D%7B0%2C3%7D+%5B%22and%22%5D+%5B%22then%22%5D+%5B%22I%22%5D+%5B%22forced%22%5D+%5B%22myself%22%5D+%5B%22walking%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22forced%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'I -- I forced -- I forced -- and then I forced myself walking . I forced . ', 'right': '', 'complete_match': 'I -- I forced -- I forced -- and then I forced myself walking . I forced . ', 'testimony_id': 'usc_shoah_2830', 'shelfmark': ['USC 2830'], 'token_start': 19635, 'token_end': 19653}


In [70]:
fragment_2 = {}
fragment_2['original_sentence'] = "So I forced myself to walk straight and to-- unbelievable."
fragment_2['label']="So I forced myself to walk straight and to-- unbelievable."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22So%22%5D+%5B%22I%22%5D+%5B%22forced%22%5D+%5B%22myself%22%5D+%5B%22to%22%5D+%5B%22walk%22%5D+%5B%22straight%22%5D+%5B%22and%22%5D+%5B%22to%22%5D+%5B%5D%7B0%2C3%7D+%5B%22unbelievable%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'So I forced myself to walk straight and to -- unbelievable . ', 'right': '', 'complete_match': 'So I forced myself to walk straight and to -- unbelievable . ', 'testimony_id': 'usc_shoah_3658', 'shelfmark': ['USC 3658'], 'token_start': 14400, 'token_end': 14412}


In [71]:
fragment_3 = {}
fragment_3['original_sentence'] = "And it was not even any physical to be outside, force yourself to walk."
fragment_3['label']="And it was not even any physical to be outside, force yourself to walk."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22it%22%5D+%5B%22was%22%5D+%5B%22not%22%5D+%5B%22even%22%5D+%5B%22any%22%5D+%5B%22physical%22%5D+%5B%22to%22%5D+%5B%22be%22%5D+%5B%22outside%22%5D+%5B%5D+%5B%22force%22%5D+%5B%22yourself%22%5D+%5B%22to%22%5D+%5B%22walk%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And it was not even any physical to be outside , force yourself to walk . ', 'right': '', 'complete_match': 'And it was not even any physical to be outside , force yourself to walk . ', 'testimony_id': 'usc_shoah_8892', 'shelfmark': ['USC 8892'], 'token_start': 21798, 'token_end': 21814}


In [72]:
fragment_4 = {}
fragment_4['original_sentence'] = "Who couldn’t walk got killed. So you make yourself walk...doesn’t matter how."
fragment_4['label']= "Who couldn’t walk got killed. So you make yourself walk...doesn’t matter how."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22Who%22%5D+%5B%22could%22%5D+%5B%5D+%5B%22walk%22%5D+%5B%22got%22%5D+%5B%22killed%22%5D+%5B%5D+%5B%22So%22%5D+%5B%22you%22%5D+%5B%22make%22%5D+%5B%22yourself%22%5D+%5B%22walk%22%5D+%5B%5D%7B0%2C50%7D+%5B%22does%22%5D+%5B%5D+%5B%22matter%22%5D+%5B%22how%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'Who could n’t walk got killed . So you make yourself walk ... does n’t matter how . ', 'right': '', 'complete_match': 'Who could n’t walk got killed . So you make yourself walk ... does n’t matter how . ', 'testimony_id': 'irn510728', 'shelfmark': ['USHMM RG-50.154*0008'], 'token_start': 10388, 'token_end': 10406}


In [73]:
add_testimonial_fragments(fragments)

### 4.  

In [74]:
lemmas = ["force","labor"]

In [75]:
query = create_contextual_query(lemmas,context_length=10)
print (query)

([lemma="force"][]{0,10}[lemma="labor"])|([lemma="labor"][]{0,10}[lemma="force"])


In [77]:
query = '([lemma="force"]["to"]{0,10}[lemma="labor"])|([lemma="labor"][]{0,10}[lemma="force"])'

In [78]:
domain_term = "work"

In [79]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [80]:
fragment_1 = {}
fragment_1['original_sentence'] = "Any kind of work assigned to the adults was of course forced. The children were forced to work as of age 10."
fragment_1['label']="Any kind of work assigned to the adults was of course forced. The children were forced to work as of age 10."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22Any%22%5D+%5B%22kind%22%5D+%5B%22of%22%5D+%5B%22work%22%5D+%5B%22assigned%22%5D+%5B%22to%22%5D+%5B%22the%22%5D+%5B%22adults%22%5D+%5B%22was%22%5D+%5B%22of%22%5D+%5B%22course%22%5D+%5B%22forced%22%5D+%5B%5D+%5B%22The%22%5D+%5B%22children%22%5D+%5B%22were%22%5D+%5B%22forced%22%5D+%5B%22to%22%5D+%5B%22work%22%5D+%5B%22as%22%5D+%5B%22of%22%5D+%5B%22age%22%5D+%5B%2210%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'Any kind of work assigned to the adults was of course forced . The children were forced to work as of age 10 . ', 'right': '', 'complete_match': 'Any kind of work assigned to the adults was of course forced . The children were forced to work as of age 10 . ', 'testimony_id': 'usc_shoah_1216', 'shelfmark': ['USC 1216'], 'token_start': 10520, 'token_end': 10544}


In [81]:
fragment_2 = {}
fragment_2['original_sentence'] = "Everybody was forced to work. And everyone was assigned in places according to their trades, according to their education."
fragment_2['label']="Everybody was forced to work. And everyone was assigned in places according to their trades, according to their education."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22Everybody%22%5D+%5B%22was%22%5D+%5B%22forced%22%5D+%5B%22to%22%5D+%5B%22work%22%5D+%5B%5D+%5B%22And%22%5D+%5B%22everyone%22%5D+%5B%22was%22%5D+%5B%22assigned%22%5D+%5B%22in%22%5D+%5B%22places%22%5D+%5B%22according%22%5D+%5B%22to%22%5D+%5B%22their%22%5D+%5B%22trades%22%5D+%5B%5D+%5B%22according%22%5D+%5B%22to%22%5D+%5B%22their%22%5D+%5B%22education%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'Everybody was forced to work . And everyone was assigned in places according to their trades , according to their education . ', 'right': '', 'complete_match': 'Everybody was forced to work . And everyone was assigned in places according to their trades , according to their education . ', 'testimony_id': 'usc_shoah_2578', 'shelfmark': ['USC 2578'], 'token_start': 1847, 'token_end': 1869}


In [82]:
fragment_3 = {}
fragment_3['original_sentence'] = "They were slaves. My sisters were forced to work. They were not, not, not paid."
fragment_3['label']="They were slaves. My sisters were forced to work. They were not, not, not paid."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22They%22%5D+%5B%22were%22%5D+%5B%22slaves%22%5D+%5B%5D+%5B%22My%22%5D+%5B%22sisters%22%5D+%5B%22were%22%5D+%5B%22forced%22%5D+%5B%22to%22%5D+%5B%22work%22%5D+%5B%5D+%5B%22They%22%5D+%5B%22were%22%5D+%5B%22not%22%5D+%5B%5D+%5B%22not%22%5D+%5B%5D+%5B%22not%22%5D+%5B%22paid%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'They were slaves . My sisters were forced to work . They were not , not , not paid . ', 'right': '', 'complete_match': 'They were slaves . My sisters were forced to work . They were not , not , not paid . ', 'testimony_id': 'usc_shoah_8725', 'shelfmark': ['USC 8725'], 'token_start': 30704, 'token_end': 30724}


In [83]:
fragment_4 = {}
fragment_4['original_sentence'] = "People were being forced to work in substandard conditions, with less than normal food."
fragment_4['label']= "People were being forced to work in substandard conditions, with less than normal food."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22People%22%5D+%5B%22were%22%5D+%5B%22being%22%5D+%5B%22forced%22%5D+%5B%22to%22%5D+%5B%22work%22%5D+%5B%22in%22%5D+%5B%22substandard%22%5D+%5B%22conditions%22%5D+%5B%5D+%5B%22with%22%5D+%5B%22less%22%5D+%5B%22than%22%5D+%5B%22normal%22%5D+%5B%22food%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'People were being forced to work in substandard conditions , with less than normal food . ', 'right': '', 'complete_match': 'People were being forced to work in substandard conditions , with less than normal food . ', 'testimony_id': 'usc_shoah_1799', 'shelfmark': ['USC 1799'], 'token_start': 13300, 'token_end': 13316}


In [84]:
fragment_5 = {}
fragment_5['original_sentence'] = "I was there for three years. I was forced to work 10 and 12 hours a day in this plane factory."
fragment_5['label']= "I was there for three years. I was forced to work 10 and 12 hours a day in this plane factory."
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22I%22%5D+%5B%22was%22%5D+%5B%22there%22%5D+%5B%22for%22%5D+%5B%22three%22%5D+%5B%22years%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22forced%22%5D+%5B%22to%22%5D+%5B%22work%22%5D+%5B%2210%22%5D+%5B%22and%22%5D+%5B%2212%22%5D+%5B%22hours%22%5D+%5B%22a%22%5D+%5B%22day%22%5D+%5B%22in%22%5D+%5B%22this%22%5D+%5B%22plane%22%5D+%5B%22factory%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'I was there for three years . I was forced to work 10 and 12 hours a day in this plane factory . ', 'right': '', 'complete_match': 'I was there for three years . I was forced to work 10 and 12 hours a day in this plane factory . ', 'testimony_id': 'HVT-180', 'shelfmark': ['Fortunoff HVT-180'], 'token_start': 3254, 'token_end': 3277}


In [85]:
add_testimonial_fragments(fragments)